In [ ]:

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

from tensorflow.keras import backend as K

def attention_block(inputs, skip_connection):

    # Compute channels for skip and output tensor
    channels = K.int_shape(skip_connection)[-1]
    filters = channels // 4

    # Compute attention mechanism
    x = Conv2D(filters, kernel_size=3, padding='same')(inputs)
    skip = Conv2D(filters, kernel_size=3, padding='same')(skip_connection)
    x = Activation('leaky_relu')(x + skip)
    x = Conv2D(1, kernel_size=3, padding='same')(x)
    x = Activation('sigmoid')(x)
    x = Multiply()([inputs, x])
    return x

def attention_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2D(512, 2, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(drop5))
    att6 = attention_block(conv4, up6)
    merge6 = concatenate([att6, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(conv6))
    att7 = attention_block(conv3, up7)
    merge7 = concatenate([att7, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(conv7))
    att8 = attention_block(conv2, up8)
    merge8 = concatenate([att8, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(conv8))
    att9 = attention_block(conv1, up9)
    merge9 = concatenate([att9, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='leaky_relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(n_classes, 1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=conv9)

    return model



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = attention_unet(input_shape=(512, 512, 1), n_classes=1)

In [ ]:
def dice_coefficient(y_true, y_pred):

  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred)
  return numerator / (denominator + tf.keras.backend.epsilon())

def dice_loss(y_true, y_pred):
  return 1 - dice_coefficient(y_true, y_pred)

In [ ]:
def fitness(x_train, y_train, model):
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
  loss=dice_loss,
  metrics=[dice_coefficient])
  history = model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=0)
  return history.history['val_dice_coefficient'][-1]

In [ ]:
def mtsa(x_train, y_train, model, num_tries, t_init, t_min, alpha):

  n = x_train.shape[0]
  d = x_train.shape[1:]
  x_best = np.random.rand(*d)
  y_best = np.random.rand(*d)
  f_best = fitness(x_train, y_train, model)
  for i in range(num_tries):
    t = t_init
    x_curr = np.random.rand(*d)
    y_curr = np.random.rand(*d)
    f_curr = fitness(x_train, y_train, model)
    while t > t_min:

      for j in range(10):

        x_prop = x_curr + np.random.normal(size=d, scale=0.1)
        y_prop = y_curr + np.random.normal(size=d, scale=0.1)
        f_prop = fitness(x_train, y_train, model)
        delta_f = f_prop - f_curr
        if delta_f < 0 or np.exp(-delta_f/t) > np.random.rand():
          x_curr = x_prop
          y_curr = y_prop
          f_curr = f_prop
        if f_curr < f_best:
          x_best = x_curr
          y_best = y_prop
          f_best = f_curr
          t *= alpha
  return x_best, y_best, f_best

In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

#EDIT HERE##############################
datafolder='/content/drive/MyDrive/11 April 2023/processeddata/'
weightsfolder='/content/drive/MyDrive/11 April 2023/modelweights'
noduleimages=np.load("/content/drive/MyDrive/10_april_2023/new_500/nod_im_1.npy")
nodulemasks=np.load("/content/drive/MyDrive/10_april_2023/new_500/nod_mask_1.npy")



In [ ]:
noduleimages=noduleimages.reshape(noduleimages.shape[0],512,512,1)
nodulemasks=nodulemasks.reshape(nodulemasks.shape[0],512,512,1)
imagestrain, imagestest, maskstrain, maskstest = train_test_split(noduleimages,nodulemasks,test_size=.20)

####
maskstrain2 = maskstrain.astype(np.float32)
maskstest2 = maskstest.astype(np.float32)

In [ ]:
x_train=imagestrain
y_train=maskstrain2

x_val=imagestest
y_val=maskstest2

In [ ]:
#normalize images between 0 and 1

xx_train=(x_train-np.min(x_train))/(np.max(x_train)-np.min(x_train))
xx_val=(x_val-np.min(x_val))/(np.max(x_val)-np.min(x_val))


#conver binary images mask into real values of nodule pixels

yy_train=x_train*y_train
yy_val=x_val*y_val

In [1]:
#x_best, y_best, f_best = mtsa(xx_train, yy_train, model, num_tries=10, t_init=10.0, t_min=0.1, alpha=0.99)



In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
loss=dice_loss,
metrics=[dice_coefficient])


In [ ]:
model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)